In [1]:
import tensorflow as tf

D:\software\Anaconda\envs\tensorflow\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
#  TF程序的两步： ①自己构建计算图   ②利用tf.Session 执行计算图中的计算流程
#  或者利用default_graph
#  一个小trick  小写的代表op  大写的代表class 一个class包含多个op
g1 = tf.Graph()
g2 = tf.get_default_graph()

with g1.as_default():
    a = tf.constant(3)
    b = tf.constant(4)
    c = tf.add(a, b)
with g2.as_default():
    d = tf.constant(4)
    e = tf.constant(4)
    f = tf.add(d, e)

with tf.Session(graph=g1) as sess:
    print(sess.run(a))
    
with tf.Session(graph=g2) as sess:
    print(sess.run(f))    

3
8


In [13]:
a = tf.constant(2)
b = tf.constant(3)
c = tf.add(a,b)

writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())
with tf.Session() as sess:
    print(sess.run(c))
writer.close()

5


In [54]:
#   通过get_variable 方法创建变量， 也可以直接通过
# W = tf.get_variable('big_matrixx', shape=(784, 10), initializer=tf.zeros_initializer())

#   初始化变量
# with tf.Session() as sess:
#     print(sess.run(W.initializer))

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     sess.run(W)

w = tf.Variable(10)
w.assign(20)  # assign 复制方法会产生一个新的Tensor
# 下面将变量w的操作存储
#  每个Session 有自己的变量空间，以及这些变量对应的操作op
w_assign_op =  w.assign(100)
w_assign_op_double = w.assign(2*w)
with tf.Session() as sess:
    sess.run(w.initializer)
#     sess.run(w_assign_op)
    sess.run(w_assign_op_double)
    sess.run(w_assign_op_double)
    print(w.eval())
    print(w_assign_op.eval())
    print(w.eval())

sess2 = tf.Session()
sess2.run(w.initializer)
print(sess2.run(w.assign_add(2)))


40
100
100
None
12


In [58]:
#   利用placeholder来存储变量
#   使用placeholder的变量的时候，要使用feed_dict 给变量复制
a = tf.placeholder(tf.float32, shape=[3])
b = tf.constant([5,5,5], tf.float32)

c = a + b
with tf.Session() as sess:
    print(sess.run(c, feed_dict={a:[1,2,3]}))
    print(sess.run(c, feed_dict={a:[1,1,1]}))

[ 6.  7.  8.]
[ 6.  6.  6.]


In [3]:
help(tf.data)

Help on package tensorflow.data in tensorflow:

NAME
    tensorflow.data - `tf.data.Dataset` API for input pipelines.

DESCRIPTION
    See @{$guide/datasets$Importing Data} for an overview.

PACKAGE CONTENTS


FILE
    c:\users\ibm\appdata\roaming\python\python35\site-packages\tensorflow\data\__init__.py




In [2]:
tf.__version__

'1.9.0'

In [ ]:
#### TF control flow
`control flow ops`: tf.group, tf.count_up_to, tf.cond, tf.case, tf.while_llop, .... <br>
`comparsion ops`:   tf.equal, tf.not_equal, tf.less, tf.greater, tf.where, ...      <br>
`logical ops`:      tf.logical_and, tf.logical_not, tf.logical_or, tf.logical_xor...<br>
`debugging ops`:    tf.is_finite, tf.is_inf, tf.is_nan, tf.Assert, tf.Print ....    <br> 

#### placeholder
优点： 容易操作，方便处理
缺点： 容易造成性能瓶颈

####   利用tf.data.DataSet 来加载和存储数据
tf.data 和 feed_dict 的比较： 利用dict来feed更加简单（python的形式），当数据量不大的时候， 建议采用


In [48]:
#  变量共享的问题 problem when Variable sharing
# def two_hidden_layers(x):
#     w1 = tf.Variable(tf.random_normal([100, 50]), name='h1_weights')
#     b1 = tf.Variable(tf.zeros([50]), name='h1_biases')
#     h1 = tf.matmul(x, w1) + b1
    
#     w2 = tf.Variable(tf.random_normal([50, 10]), name='h2_weights')
#     b2 = tf.Variable(tf.zeros([10]), name='h2_biases')
#     logits = tf.matmul(h1, w2) + b2
#     return logits
x1 = tf.random_normal([50, 100])
x2 = tf.random_normal([50, 100])
# logits1 = two_hidden_layers(x1)
# logits2 = two_hidden_layers(x2)

# print(logits1[0][0].get, logits2[0][0])

# 通过get_variable 来共享变量
# def two_hidden_layers_share(x):
#     assert x.shape.as_list() == [200, 100]
#     w1 = tf.get_variable('h1_weights', [100, 50], initializer=tf.random_normal_initializer())
#     b1 = tf.get_variable('h1_biases', [50], initializer=tf.constant_initializer(0.0))
#     h1 = tf.matmul(x, w1) + b1
     
#     assert h1.shape.as_list() == [200, 50]
#     w2 = tf.get_variable('h2_weights', [50, 10], initializer=tf.random_normal_initializer())
#     b2 = tf.get_variable('b2_biases', [10], initializer=tf.constant_initializer(0.0))
#     logits = tf.matmul(h1, w2) + b2
#     return logits
#  如果直接这么共享变量的话，会出现ValueError:Variable h1_weights already exists 之类的错误
# logits1 = two_hidden_layers_share(x1)
# logits2 = two_hidden_layers_share(x2)

# 如下，将共享的变量放入一个scope当中，在这个scope中复用这些变量
# with tf.variable_scope('two_layers', reuse=tf.AUTO_REUSE) as scope:
#     logits1 = two_hidden_layers_share(x1)
#     scope.reuse_variables()
#     logits2 = two_hidden_layers_share(x2)
#     print(logits1[0][0], logits2[0][0])
   


# 通过在函数中定义 variable_scope 来定义不同的变量空间  来更好的复用变量
# def fully_connected(x, output_dim, scope):
#     with tf.variable_scope(scope, reuse=tf.AUTO_REUSE) as scope:
#         w = tf.get_variable('weights', [x.shape[1], output_dim], initializer=tf.random_normal_initializer())
#         b = tf.get_variable('biases', [output_dim], initializer=tf.constant_initializer(0.0))
#         return tf.matmul(x, w) + b

# def two_hidden_layers(x):
#     h1 = fully_connected(x, 50, 'h1')
#     h2 = fully_connected(h1, 10, 'h2')

# with tf.variable_scope('two_layers') as scope:
#     logits1 = two_hidden_layers(x1)
#     logits2 = two_hidden_layers(x2)
    
# tf.train.Saver 保存和存储变量
# 保存最近的latest检查点





# 随机性random  根据random的seed不同可以产生不同的随机值
# def produceRandom(seedNumber):
#     return tf.random_uniform([], -10, 10, seed=seedNumber)

# c = tf.random_uniform([], -10, 10, seed=2)

# with tf.Session() as sess:
#     print(sess.run(produceRandom(2)))
#     print(sess.run(produceRandom(3)))
    
# with tf.Session() as sess:
#     print(sess.run(c))
# with tf.Session() as sess:
#     print(sess.run(c))    


# 利用 tf.gradients() 计算偏导
x = tf.Variable(2.0)
y = 2.0 * (x**3)
z = 3.0 + y ** 2
grad_z = tf.gradients(z, [x, y])
with tf.Session()  as sess:
    sess.run(x.initializer)
    print(sess.run(grad_z))

[768.0, 32.0]


In [47]:
help(tf.gradients)

Help on function gradients in module tensorflow.python.ops.gradients_impl:

gradients(ys, xs, grad_ys=None, name='gradients', colocate_gradients_with_ops=False, gate_gradients=False, aggregation_method=None, stop_gradients=None)
    Constructs symbolic derivatives of sum of `ys` w.r.t. x in `xs`.
    
    `ys` and `xs` are each a `Tensor` or a list of tensors.  `grad_ys`
    is a list of `Tensor`, holding the gradients received by the
    `ys`. The list must be the same length as `ys`.
    
    `gradients()` adds ops to the graph to output the derivatives of `ys` with
    respect to `xs`.  It returns a list of `Tensor` of length `len(xs)` where
    each tensor is the `sum(dy/dx)` for y in `ys`.
    
    `grad_ys` is a list of tensors of the same length as `ys` that holds
    the initial gradients for each y in `ys`.  When `grad_ys` is None,
    we fill in a tensor of '1's of the shape of y for each y in `ys`.  A
    user can provide their own initial `grad_ys` to compute the
    deriva